# Pairwise alignmnet visualization for all the most seen sequence in each timespan
First, get our 10 rows with the most seen seq for each timespan

In [1]:
import pandas as pd
import os

project_folder = os.path.join("..", "..")

In [3]:
df = pd.read_csv(os.path.join(project_folder, 'data', 'final', "all.csv"))
df.head(1)

,state,p_sequence,p_accession,date,count,n_accession,n_sequence,clade,timespan
0,MA,"""MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QTP71261,2020,2,MW885877,GGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTA...,20A,1


In [22]:
temp = df.groupby('timespan', as_index = False).max('count')
top_count_clade_per_timespan_df = df.merge(temp, on = ['timespan', 'count']).sort_values('timespan')

In [23]:
top_count_clade_per_timespan_df.head(10)

,state,p_sequence,p_accession,date,count,n_accession,n_sequence,clade,timespan
0,WA,"""MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QLJ57227,2020,1367,MT252714,CTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGT...,19B,1
1,WA,"""MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QLJ57227,2020,1129,MT252714,CTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGT...,19B,2
2,CA,"""MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QTX16612,2020-05,187,MW940521,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,20C,3
3,SD,"""MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QWE90634,2020-09-23,297,MZ346443,ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...,20G,4
4,VA,"""MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QTN63407,2021-01,3558,MW868904,ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...,20C,5
5,TX,"""MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QVR40162,2/1/21,3755,MZ267386,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,20G,6
6,OH,"""MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QVO59492,3/1/21,499,MZ256071,TAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATC...,20G,7
7,NY,"""MFVFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QUQ04393,2021-04-01,1005,MZ082876,AAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCT...,21F (Iota),8
8,VA,"""MFVFLVLLPLVSSQCVNFTNRTQLPSAYTNSFTRGVYYPDKVFRS...",QWY94091,2021-05,1509,MZ451365,ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...,"20J (Gamma, V3)",9
9,VA,"""MFVFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRS...",QXP14639,2021-06,2493,MZ571287,TTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTG...,"20I (Alpha, V1)",10


In [17]:
from Bio import Align
from Bio.Align import substitution_matrices
import plotly.graph_objects as go
import math

In [131]:
def plot_alignment_plotly(alignments, target_row, query_row, save = False, debug = False):
    string_alignments = alignments[0].format().split('\n')
    target = string_alignments[0]
    match_type = string_alignments[1]
    query = string_alignments[2]
    y_height = 1
    length = len(target)
    if debug:
        print(length)
        
    x_insert =[None]
    x_delete =[None]
    x_sub_low =[None]
    x_sub_high =[None] # none means it will show in legend even when there is no data

    labels = []
    yshift = 18
    level = 0
    for i in range(0, length):
        if match_type[i] == '.':
            # substitution, low conservation
            x_sub_low.append(i)
            labels.append({"text":target[i] + str(i) + query[i], 'x':i, 'y':y_height , 
                           'font' :{'size': 12}, 'showarrow':False, 'yshift': yshift * (level % 2 + 1), 'yanchor':'middle', 'xanchor':'center'})
           
            if yshift < 0:
                level = level + 1
            yshift = -yshift
        elif match_type[i] == ':':
            # subsitution, high conservation
            x_sub_high.append(i)
            labels.append({"text":target[i] + str(i) + query[i], 'x' :i,  'y':y_height , 
                           'font' :{'size': 12}, 'showarrow':False, 'yshift': yshift * (level % 2 + 1), 'yanchor':'middle', 'xanchor':'center'})
            if yshift < 0:
                level = level + 1
            yshift = -yshift
        elif match_type[i] == '-':
            # deletion or insertion
            if target[i] =='-':
                # insertion
                x_insert.append(i)
                labels.append({"text":str(i) + query[i], 'x' :i,  'y':y_height , 
                           'font' :{'size': 12}, 'showarrow':False, 'yshift': yshift * (level % 2 + 1), 'yanchor':'middle', 'xanchor':'center'})
                if yshift < 0:
                    level = level + 1
                yshift = -yshift
            else: 
                # deletion (alg not allowed to match gap to gap)
                x_delete.append(i)
                labels.append({"text": target[i] + str(i), 'x' :i, 'y':y_height , 
                           'font' :{'size': 12}, 'showarrow':False, 'yshift': yshift * (level % 2 + 1), 'yanchor':'middle', 'xanchor':'center'})
                if yshift < 0:
                    level = level + 1
                yshift = -yshift
                
    total = len(x_insert)+ len(x_delete) +len(x_sub_low) + len(x_sub_high) - 4 # total number of changes
    counts = [len(x_insert)-1, len(x_delete) - 1, len(x_sub_low) -1 , len(x_sub_high) - 1 ]
    names = ['Insertions', 'Deletions', 'Substitution, semi-conservative', 'Substitution, conservative']
    
    if debug:
        print(total)

    fig = go.Figure(
        [go.Scatter( # base gray line
            mode = "lines",
            x =  [0, length],
            y = [y_height, y_height], 
            name = "sequence line",
            line = {'color':  'gray'},
            showlegend = False,
            hoverinfo = 'skip'
        ),
         go.Scatter( # S1  line
            mode = "lines",
            x =  [14, 685],
            y = [y_height, y_height], 
            name = "S1",
            line = {'color':  'plum', 'width' : 3},
            legendgroup = 'Subunit',
            showlegend = True,
            hoverinfo = 'skip'
        ),
         go.Scatter( # S2  line
            mode = "lines",
            x =  [686, 1273],
            y = [y_height, y_height], 
            name = "S2",
            line = {'color':  'pink', 'width':3},
            legendgroup = 'Subunit',
            showlegend = True,
            hoverinfo = 'skip'
        ),
        go.Scatter( # insert markers
            mode = "markers",
            x = x_insert ,
            y = [y_height for y in x_insert], 
            name = "insertion",
            marker=dict(size=8, color = 'orange', symbol = 'square'),
            legendgroup = 'changes',
            showlegend = True,
            hoverinfo = 'x'
        ),
          go.Scatter( # delete markers
            mode = "markers",
            x = x_delete,
            y = [y_height for y in x_delete], 
            name = "deletion",
            marker=dict(size=8, color = 'red', symbol = 'square'),
            legendgroup = 'changes',
            showlegend = True,
            hoverinfo = 'x'
        ),
         go.Scatter( # sub semi markers
            mode = "markers",
            x = x_sub_low ,
            y = [y_height for y in x_sub_low], 
            name = "substitution, semi-conservative",
            marker=dict(size=8, color = 'blue', symbol = 'circle'),
            legendgroup = 'changes',
             showlegend = True,
            hoverinfo = 'x'
        ),
          go.Scatter( # sub conservative markers
            mode = "markers",
            x = x_sub_high ,
            y = [y_height for y in x_sub_high], 
            name = "substitution, conservative",
            marker=dict(size=8, color = 'lightblue', symbol = 'circle'),
            legendgroup = 'changes',
            showlegend = True,
            hoverinfo = 'x'
        ),
          go.Bar( # score bar
             x = [length, (alignments[0].score / 6722) * length], # best score is when all positions match, match contributes score of 4
                                         # could use log score, got 6722 when matched sequence with itself
             y = [0.08, 0.08],
             orientation = 'h',
             showlegend = False,
             width = 0.16,
             text = [None, int(alignments[0].score)], # score should always be int, we don't need decimal precision and don't want .0 cluttering
             textposition = ['inside', "outside"],
             marker_color =['lightgray', 'lightgray'],
             marker = dict(pattern = {'bgcolor':['lightgray', 'gray'], 'shape':['', '/'], 'size':5}),
             hoverinfo = 'skip'
         ) ,
        go.Bar( # counts bars
             name = "counts",
             x = [1540, 1590, 1640,1700], 
             y = [(c/total) *0.85 for c in counts ],
             showlegend = False,
             width = 30,
             text =  counts, 
             marker_color =['orange', 'red', 'blue', 'lightblue'],
             marker = {'line' :dict(width = 2, color = ['orange','red','blue','lightblue'])},
             textposition = 'outside',
             hoverinfo = 'skip' # can't get the names without them in x,y,text
         )

        ]
    )

    fig.update_xaxes(
            range = [-15,1850],
            showgrid=False, # thin lines in the background
            zeroline= False, # thick line at x=0
            visible= False  # numbers below
            )

    fig.update_yaxes(
            range =[0,1.5],
            showgrid= False, # thin lines in the background
            zeroline= False, # thick line at x=0
            visible= False  # numbers below
            )
    
    labels.append({'x': 0, 'y':0.25 ,'text': "Alignment Score" ,'font': { 'size': 12},'showarrow':False, 'xanchor':'left', 'yanchor':'bottom' })
    labels.append({'x':1450, 'y':0, 'text':'counts', 'yanchor':'bottom','showarrow':False, 'textangle':0})
    subtitle = "{0} (T{4}, {1}) difference from {2} (T{5}, {3})".format(target_row['p_accession'], target_row['clade'], query_row['p_accession'], query_row['clade'], target_row['timespan'], query_row['timespan'])
    button_layer_1_height = 0.7
    fig.update_layout(plot_bgcolor='white',
                      height=380,
                      width = 1100,
                      barmode = 'overlay',
                      title = dict(text = "Spike Protien Amino Acid Sequence Comparison<br><sup>{0}<sup>".format(subtitle),
                                   x = 0.1
                                  ),
                      legend=dict(
                                yanchor="middle",
                                y=1,
                                xanchor="right",
                                x=1
                    ),
                      annotations= labels,
                      
                      updatemenus=[
                                dict(
                                    buttons=list([
                                        dict(
                                            args=["Target", "T1"],
                                            label="T1",
                                            method="restyle"
                                        ),
                                        dict(
                                            args=["Target", "T2"],
                                            label="T2",
                                            method="restyle"
                                        ),
                                        dict(
                                            args=["Target", "T3"],
                                            label="T2",
                                            method="restyle"
                                        )   
                                    ]),
                                    direction = 'down',
                                    pad={"r": 10, "t": 10},
                                    showactive=True,
                                    x=0.52,
                                    xanchor="left",
                                    y=1.4,
                                    yanchor="top"
                                    
                                ),
                                  dict(
                                    buttons=list([
                                        dict(
                                            args=["Target", "T1"],
                                            label="T1",
                                            method="relayout",
                                            args=["shapes"]
                                        ),
                                        dict(
                                            args=["Target", "T2"],
                                            label="T2",
                                            method="restyle"
                                        ),
                                        dict(
                                            args=["Target", "T3"],
                                            label="T2",
                                            method="restyle"
                                        )   
                                    ]),
                                    direction = 'down',
                                    pad={"r": 10, "t": 10},
                                    showactive=True,
                                    x=0.62,
                                    xanchor="left",
                                    y=1.4,
                                    yanchor="top"
                                    
                                )]
    )
    
    fig.show()
    if save:
        fig.write_html('test2.html')
        fig.write_image('test2.png')

In [132]:
def graph(top_count_clade_per_timespan_df, timespan1, timespan2):
    aligner = Align.PairwiseAligner(match = 4,
         mismatch = -1, 
         target_open_gap_score = -1000, 
         target_extend_gap_score  = -1000,
         query_open_gap_score = -20,
         query_extend_gap_score = -4) # parameters from this paper https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7314508/

    substitution_matrices.load() 
    matrix = substitution_matrices.load("BLOSUM62")
    aligner.substitution_matrix = matrix

    seq1 = top_count_clade_per_timespan_df.loc[timespan1 - 1]
    seq2 = top_count_clade_per_timespan_df.loc[timespan2 - 1]
    alignments = aligner.align(seq1['p_sequence'][1:-1], seq2['p_sequence'][1:-1])
    plot_alignment_plotly(alignments, df.loc[df['p_accession'] == seq1['p_accession']].to_dict('records')[0], df.loc[df['p_accession'] == seq2['p_accession']].to_dict('records')[0], save = False, debug = True)

In [133]:
graph(top_count_clade_per_timespan_df,1,10)

1273
7


So the top ones are not that different. May want to try most common of each clade to see more difference. First lets get this working with the interactive choice of which two clades to compare. or compare the three deltas

In [277]:
def make_aligner():
    aligner = Align.PairwiseAligner(match = 4,
         mismatch = -1, 
         target_open_gap_score = -1000, 
         target_extend_gap_score  = -1000,
         query_open_gap_score = -20,
         query_extend_gap_score = -4) # parameters from this paper https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7314508/

    substitution_matrices.load() 
    matrix = substitution_matrices.load("BLOSUM62")
    aligner.substitution_matrix = matrix
    return aligner

def get_relevent_seqs(top_count_clade_per_timespan_df,timespan1, timespan2):
    seq1 = top_count_clade_per_timespan_df.loc[timespan1 - 1]
    seq2 = top_count_clade_per_timespan_df.loc[timespan2 - 1]
    alignments = aligner.align(seq1['p_sequence'][1:-1], seq2['p_sequence'][1:-1])
    target_row = df.loc[df['p_accession'] == seq1['p_accession']].to_dict('records')[0]
    query_row =df.loc[df['p_accession'] == seq2['p_accession']].to_dict('records')[0]
    return alignments, target_row, query_row
    
def calc_traces_labels_and_subtitle(top_count_clade_per_timespan_df,timespan1, timespan2, debug = False):
    alignments, target_row, query_row = get_relevent_seqs(top_count_clade_per_timespan_df,timespan1, timespan2)
    string_alignments = alignments[0].format().split('\n')
    target = string_alignments[0]
    match_type = string_alignments[1]
    query = string_alignments[2]
    y_height = 1
    length = len(target)
    if debug:
        print(length)
        
    x_insert =[None]
    x_delete =[None]
    x_sub_low =[None]
    x_sub_high =[None] # none means it will show in legend even when there is no data

    labels = []
    yshift = 18
    level = 0
    for i in range(0, length):
        if match_type[i] == '.':
            # substitution, low conservation
            x_sub_low.append(i)
            labels.append({"text":target[i] + str(i) + query[i], 'x':i, 'y':y_height , 
                           'font' :{'size': 12}, 'showarrow':False, 'yshift': yshift * (level % 2 + 1), 'yanchor':'middle', 'xanchor':'center'})
           
            if yshift < 0:
                level = level + 1
            yshift = -yshift
        elif match_type[i] == ':':
            # subsitution, high conservation
            x_sub_high.append(i)
            labels.append({"text":target[i] + str(i) + query[i], 'x' :i,  'y':y_height , 
                           'font' :{'size': 12}, 'showarrow':False, 'yshift': yshift * (level % 2 + 1), 'yanchor':'middle', 'xanchor':'center'})
            if yshift < 0:
                level = level + 1
            yshift = -yshift
        elif match_type[i] == '-':
            # deletion or insertion
            if target[i] =='-':
                # insertion
                x_insert.append(i)
                labels.append({"text":str(i) + query[i], 'x' :i,  'y':y_height , 
                           'font' :{'size': 12}, 'showarrow':False, 'yshift': yshift * (level % 2 + 1), 'yanchor':'middle', 'xanchor':'center'})
                if yshift < 0:
                    level = level + 1
                yshift = -yshift
            else: 
                # deletion (alg not allowed to match gap to gap)
                x_delete.append(i)
                labels.append({"text": target[i] + str(i), 'x' :i, 'y':y_height , 
                           'font' :{'size': 12}, 'showarrow':False, 'yshift': yshift * (level % 2 + 1), 'yanchor':'middle', 'xanchor':'center'})
                if yshift < 0:
                    level = level + 1
                yshift = -yshift
                
    total = len(x_insert)+ len(x_delete) +len(x_sub_low) + len(x_sub_high) - 4 # total number of changes
    if total == 0:
        # same sequence /exact match
        labels.append({'x':680, 'y':1, 'text':'Same Sequence', 'yanchor':'bottom','showarrow':False, 'textangle':0, 'font':{'size':16}})
        total = 1 # exact match, won't affect bars because each is 0, do want to avoid division by zero
    counts = [len(x_insert)-1, len(x_delete) - 1, len(x_sub_low) -1 , len(x_sub_high) - 1 ]
    names = ['Insertions', 'Deletions', 'Substitution, semi-conservative', 'Substitution, conservative']
    
    if debug:
        print(total)

    
    traces =  [go.Scatter( # base gray line
            mode = "lines",
            x =  [0, length],
            y = [y_height, y_height], 
            name = "sequence line",
            line = {'color':  'gray'},
            showlegend = False,
            hoverinfo = 'skip',
            visible = False
        ),
         go.Scatter( # S1  line
            mode = "lines",
            x =  [14, 685],
            y = [y_height, y_height], 
            name = "S1",
            line = {'color':  'plum', 'width' : 3},
            legendgroup = 'Subunit',
            showlegend = True,
            hoverinfo = 'skip',
            visible = False
        ),
         go.Scatter( # S2  line
            mode = "lines",
            x =  [686, 1273],
            y = [y_height, y_height], 
            name = "S2",
            line = {'color':  'pink', 'width':3},
            legendgroup = 'Subunit',
            showlegend = True,
            hoverinfo = 'skip',
            visible = False
        ),
        go.Scatter( # insert markers
            mode = "markers",
            x = x_insert ,
            y = [y_height for y in x_insert], 
            name = "insertion",
            marker=dict(size=8, color = 'orange', symbol = 'square'),
            legendgroup = 'changes',
            showlegend = True,
            hoverinfo = 'x',
            visible = False
        ),
          go.Scatter( # delete markers
            mode = "markers",
            x = x_delete,
            y = [y_height for y in x_delete], 
            name = "deletion",
            marker=dict(size=8, color = 'red', symbol = 'square'),
            legendgroup = 'changes',
            showlegend = True,
            hoverinfo = 'x',
            visible = False
        ),
         go.Scatter( # sub semi markers
            mode = "markers",
            x = x_sub_low ,
            y = [y_height for y in x_sub_low], 
            name = "substitution, semi-conservative",
            marker=dict(size=8, color = 'blue', symbol = 'circle'),
            legendgroup = 'changes',
             showlegend = True,
            hoverinfo = 'x',
            visible = False
        ),
          go.Scatter( # sub conservative markers
            mode = "markers",
            x = x_sub_high ,
            y = [y_height for y in x_sub_high], 
            name = "substitution, conservative",
            marker=dict(size=8, color = 'lightblue', symbol = 'circle'),
            legendgroup = 'changes',
            showlegend = True,
            hoverinfo = 'x',
            visible = False
        ),
          go.Bar( # score bar
             x = [length, (alignments[0].score / 6722) * length], # best score is when all positions match, match contributes score of 4
                                         # could use log score, got 6722 when matched sequence with itself
             y = [0.08, 0.08],
             orientation = 'h',
             showlegend = False,
             width = 0.16,
             text = [None, int(alignments[0].score)], # score should always be int, we don't need decimal precision and don't want .0 cluttering
             textposition = ['inside', "outside"],
             marker_color =['lightgray', 'lightgray'],
             marker = dict(pattern = {'bgcolor':['lightgray', 'gray'], 'shape':['', '/'], 'size':5}),
             hoverinfo = 'skip',
            visible = False
         ) ,
        go.Bar( # counts bars
             name = "counts",
             x = [1540, 1590, 1640,1700], 
             y = [(c/total) *0.85 for c in counts ],
             showlegend = False,
             width = 30,
             text =  counts, 
             marker_color =['orange', 'red', 'blue', 'lightblue'],
             marker = {'line' :dict(width = 2, color = ['orange','red','blue','lightblue'])},
             textposition = 'outside',
             hoverinfo = 'skip', # can't get the names without them in x,y,text
            visible = False
         )

    ]
    
    labels.append({'x': 0, 'y':0.25 ,'text': "Alignment Score" ,'font': { 'size': 12},'showarrow':False, 'xanchor':'left', 'yanchor':'bottom' })
    labels.append({'x':1450, 'y':0, 'text':'counts', 'yanchor':'bottom','showarrow':False, 'textangle':0})
    subtitle = "{2} (T{5}, {3}) difference from {0} (T{4}, {1})".format(target_row['p_accession'], target_row['clade'], query_row['p_accession'], query_row['clade'], target_row['timespan'], query_row['timespan'])
    title = "Spike Protien Comparison<br><sup>of Most Frequent Sequence of Selected Timespan to First Timespan<br>{0}<sup>".format(subtitle)

    return [dict(data = traces), dict(annotations = labels, title = dict(text = title, x = 0.1))]
        

def plot_alignment_plotly(top_count_clade_per_timespan_df, save = False, debug = False):
    aligner = make_aligner()
    pairwise = dict()
    t1 = 1
    
    # t2 as special case so can switch visible to true so we see it when graph origonal renders
    pairwise[2] = calc_traces_labels_and_subtitle(top_count_clade_per_timespan_df, t1, 2)
    traces =  pairwise[2][0]['data']
    for trace in traces:
        trace.update(visible = True)
    for t2  in range(3,11):
        pairwise[t2] = calc_traces_labels_and_subtitle(top_count_clade_per_timespan_df, t1, t2)
        traces = traces + pairwise[t2][0]['data']
            
    states = pairwise[2]
    t2 = 2
    def get_state(pairwise, query = None):
        if query is None:
            return states
        elif query:
            t2 = query
        states = pairwise[t2]

        visible = []
        for t in range(1,11):
            if t == t2 - 1: #-1 because no row for timespan 1
                visible = visible + [True, True, True, True, True, True, True, True, True]
            else:
                visible = visible + [False, False,False,False,False,False,False,False,False]
        return [dict(visible = visible),states[1]]
    
    
    
    fig = go.Figure(traces)

    fig.update_xaxes(
            range = [-15,1850],
            showgrid=False, # thin lines in the background
            zeroline= False, # thick line at x=0
            visible= False  # numbers below
            )

    fig.update_yaxes(
            range =[0,1.5],
            showgrid= False, # thin lines in the background
            zeroline= False, # thick line at x=0
            visible= False  # numbers below
            )
    
    button_layer_1_height = 0.7
    fig.update_layout(plot_bgcolor='white',
                      height=380,
                      width = 1100,
                      barmode = 'overlay',
                      title = states[1]['title'],
                      legend=dict(
                                yanchor="middle",
                                y=1,
                                xanchor="right",
                                x=1
                    ),
                      annotations= states[1]['annotations'],
                      
                      updatemenus=[
                               
                                  dict(
                                    buttons=list([
                                        dict(
                                            label="T2",
                                             method="update",
                                             args = get_state(pairwise,query = 2)
                                        ),
                                        dict(
                                            label="T3",
                                            method="update",
                                            args = get_state(pairwise,query = 3)
                                        ),
                                        dict(
                                            label="T4",
                                            method="update",
                                            args = get_state(pairwise,query = 4)
                                        ),
                                        dict(
                                            label="T5",
                                            method="update",
                                            args = get_state(pairwise,query = 5)
                                        ),
                                        dict(
                                            label="T6",
                                            method="update",
                                            args = get_state(pairwise,query = 6)
                                        ),
                                        dict(
                                            label="T7",
                                            method="update",
                                            args = get_state(pairwise,query = 7)
                                        ),
                                        dict(
                                            label="T8",
                                            method="update",
                                            args = get_state(pairwise,query = 8)
                                        ) ,
                                        dict(
                                            label="T9",
                                            method="update",
                                            args = get_state(pairwise,query = 9)
                                        )  ,
                                        dict(
                                            label="T10",
                                            method="update",
                                            args = get_state(pairwise,query = 10)
                                        )  
                                        
                                    ]),
                                    direction = 'down',
                                    pad={"r": 10, "t": 10},
                                    showactive=True,
                                    x=0.62,
                                    xanchor="left",
                                    y=1.4,
                                    yanchor="top"
                                    
                                )]
    )
    fig.show()
    if save:
        fig.write_html('../../visualizations/compare_most_frequent_seq_for_timespanx_to_timespan1.html')

In [278]:
plot_alignment_plotly(top_count_clade_per_timespan_df, save = True, debug = False)

In [275]:
# title could use some work